In [337]:
!pip install tfx

In [338]:
import os
import pprint
import tempfile
import urllib
import polars as pl
import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [339]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.15.1
TFX version: 1.15.0


In [340]:
from datetime import datetime

GOOGLE_CLOUD_REGION = 'us-central1'
GOOGLE_CLOUD_PROJECT  = 'brldi-ds-capabilities-ccai'
GCS_BUCKET_NAME = 'chicago_taxi_mlops_pipeline'

PIPELINE_NAME = 'tensorflow-pipeline'

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
# Path to various pipeline artifact.
PIPELINE_ROOT = f'gs://{GCS_BUCKET_NAME}/pipeline_root/{PIPELINE_NAME}'

# Paths for users' Python module.
MODULE_ROOT = f'gs://{GCS_BUCKET_NAME}/pipeline_module/{PIPELINE_NAME}'

# Paths for users' data.
DATA_ROOT = f'gs://{GCS_BUCKET_NAME}/data/{PIPELINE_NAME}'

# Name of Vertex AI Endpoint.
ENDPOINT_NAME = PIPELINE_NAME +'-'+ TIMESTAMP

In [341]:
from google.cloud import bigquery
query = """SELECT * FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE EXTRACT(year FROM trip_start_timestamp)>2019
"""

In [342]:
client = bigquery.Client()
data = client.query(query)
data = data.to_dataframe()
print(data.shape)

(48191, 23)


In [343]:
data["trip_start_day"] = data.trip_start_timestamp.apply(lambda x: x.day)

In [344]:
data["trip_start_month"] = data.trip_start_timestamp.apply(lambda x: x.month)

In [345]:
data["trip_start_hour"] = data.trip_start_timestamp.apply(lambda x: x.hour)

In [346]:
data.to_csv("Chicago_Taxi_From_2020.csv")

In [347]:
!gsutil mb -l {GOOGLE_CLOUD_REGION} gs://{GCS_BUCKET_NAME}
!gsutil cp Chicago_Taxi_From_2020.csv {DATA_ROOT}/

Creating gs://chicago_taxi_mlops_pipeline/...
ServiceException: 409 A Cloud Storage bucket named 'chicago_taxi_mlops_pipeline' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
Copying file://Chicago_Taxi_From_2020.csv [Content-Type=text/csv]...
/ [1 files][ 19.2 MiB/ 19.2 MiB]                                                
Operation completed over 1 objects/19.2 MiB.                                     


In [348]:
import importlib
import exampleGen

# Reload the module
importlib.reload(exampleGen)

example_gen = exampleGen.exampleGen(DATA_ROOT)

In [349]:
# artifact = example_gen.outputs['examples'].get()[0]
# print(artifact.split_names, artifact.uri)

In [350]:
# # Get the URI of the output artifact representing the training examples, which is a directory
# train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# # Get the list of files in this directory (all compressed TFRecord files)
# tfrecord_filenames = [os.path.join(train_uri, name)
#                       for name in os.listdir(train_uri)]

# # Create a `TFRecordDataset` to read these files
# dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# # Iterate over the first 3 records and decode them.
# for tfrecord in dataset.take(3):
#   serialized_example = tfrecord.numpy()
#   example = tf.train.Example()
#   example.ParseFromString(serialized_example)
#   pp.pprint(example)

In [351]:
import importlib
import statisticsGen

# Reload the module
importlib.reload(statisticsGen)

statistics_gen = statisticsGen.statisticsGen(example_gen)
# context.run(statistics_gen, enable_cache=True)

In [352]:
# context.show(statistics_gen.outputs['statistics'])

In [353]:
import importlib
import schemaGen

# Reload the module
importlib.reload(schemaGen)

schema_gen = schemaGen.schemaGen(statistics_gen)
# context.run(schema_gen, enable_cache=True)

In [354]:
# context.show(schema_gen.outputs["schema"])

In [355]:
import importlib
import exampleValidator

# Reload the module
importlib.reload(exampleValidator)

example_validator = exampleValidator.exampleValidator(statistics_gen, schema_gen)
# context.run(example_validator, enable_cache=True)

In [356]:
# context.show(example_validator.outputs['anomalies'])

In [357]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=f"gs://{GCS_BUCKET_NAME}/models/taxi_transform.py")
# context.run(transform, enable_cache=True)

In [358]:
# context.show(transform.outputs["transform_graph"])

In [359]:
# train_uri = transform.outputs['transform_graph'].get()[0].uri
# os.listdir(train_uri)

In [360]:
# # Get the URI of the output artifact representing the transformed examples, which is a directory
# train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# # Get the list of files in this directory (all compressed TFRecord files)
# tfrecord_filenames = [os.path.join(train_uri, name)
#                       for name in os.listdir(train_uri)]

# # Create a `TFRecordDataset` to read these files
# dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# # Iterate over the first 3 records and decode them.
# for tfrecord in dataset.take(3):
#   serialized_example = tfrecord.numpy()
#   example = tf.train.Example()
#   example.ParseFromString(serialized_example)
#   pp.pprint(example)

In [361]:
trainer = tfx.components.Trainer(
    module_file=f"gs://{GCS_BUCKET_NAME}/models/taxi_trainer.py",
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
# context.run(trainer, enable_cache=True)

In [362]:
# model_artifact_dir = trainer.outputs['model'].get()[0].uri
# pp.pprint(os.listdir(model_artifact_dir))
# model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
# pp.pprint(os.listdir(model_dir))

In [363]:
# model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

# %load_ext tensorboard
# %tensorboard --logdir {model_run_artifact_dir}

In [364]:
# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.

import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
    import importlib
import importlib.util
from google.cloud import storage

# Define the Cloud Storage path
bucket_name = "chicago_taxi_mlops_pipeline"
blob_name = "models/taxi_constants.py"
local_file_path = "/tmp/taxi_constants.py"  # Local path to download the file

# Download the file from Cloud Storage
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.download_to_filename(local_file_path)

# Load the module dynamically
spec = importlib.util.spec_from_file_location("taxi_constants", local_file_path)
taxi_constants = importlib.util.module_from_spec(spec)
spec.loader.exec_module(taxi_constants)

# Now you can access the LABEL_KEY attribute
label_key = taxi_constants.LABEL_KEY


eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=label_key,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(
            feature_keys=['trip_start_hour'])
    ])

In [365]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
# context.run(model_resolver, enable_cache=True)


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 20
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [366]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
# context.run(evaluator, enable_cache=True)

In [367]:
 evaluator.outputs

{'evaluation': OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'blessing': OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}

In [369]:
# import tensorflow_model_analysis as tfma

# # Get the TFMA output result path and load the result.
# PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
# tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# # Show data sliced along feature column trip_start_hour.
# tfma.view.render_slicing_metrics(
#     tfma_result, slicing_column='trip_start_hour')

In [370]:
# blessing_uri = evaluator.outputs['blessing'].get()[0].uri
# !ls -l {blessing_uri}

In [371]:
# PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
# print(tfma.load_validation_result(PATH_TO_RESULT))

In [372]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory="gs://chicago_taxi_mlops_pipeline/pusher")))
# context.run(pusher, enable_cache=True)

In [374]:
# push_uri = pusher.outputs['pushed_model'].get()[0].uri
# model = tf.saved_model.load(push_uri)

# for item in model.signatures.items():
#   pp.pprint(item)

In [376]:
components = [
    example_gen,
    statistics_gen,
    schema_gen,
    example_validator,
    transform,
    trainer,
    model_resolver,
    evaluator,
    pusher
  ]

In [377]:
pipeline = tfx.dsl.Pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      components=components)

In [379]:
  # Example name, replace with your desired name
PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '-pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE
)

In [380]:
_ = runner.run(pipeline)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying taxi_transform.py -> build/lib
installing to /var/tmp/tmp2jczwv76
running install
running install_lib
copying build/lib/taxi_transform.py -> /var/tmp/tmp2jczwv76
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmp2jczwv76/tfx_user_code_Transform-0.0+81f524a1c5c8e7c5d8afa3fa47bbb1b9952677d155fa51958bc176ffd58f6f8f-py3.10.egg-info
running install_scripts
creating /var/tmp/tmp2jczwv76/tfx_user_code_Transf

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying taxi_trainer.py -> build/lib
installing to /var/tmp/tmpbxjn66kh
running install
running install_lib
copying build/lib/taxi_trainer.py -> /var/tmp/tmpbxjn66kh
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmpbxjn66kh/tfx_user_code_Trainer-0.0+393214a367044f6d52018f533ee0f28dff5042920e5d9279e0d8c450f92d0b7f-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpbxjn66kh/tfx_user_code_Trainer-0.0+393214a367044f6d

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


In [381]:
eval_config

model_specs {
  signature_name: "serving_default"
  label_key: "tips"
  preprocessing_function_names: "transform_features"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "trip_start_hour"
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
  }
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
      change_threshold {
        absolute {
          value: -1e-10
        }
        direction: HIGHER_IS_BETTER
      }
    }
  }
}